Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your collaborators below:

In [ ]:
COLLABORATORS = ""

---

# Turing Machines
As discussed in class and in section, a Turing machine consists of the following components:

- An (infinitely long) **tape**, divided into discrete cells. 
    * Each cell contains a single symbol from a finite alphabet, e.g., the set $\{0, 1\}$. 


- A **head** which can be positioned over a single cell on the tape.
    * At each timestep, the head reads the current symbol, writes a new symbol to the current cell, and then either moves 1 cell to the left, moves one cell to the right, or halts.


- A **register** that stores the current internal state of the machine. 
    * At any given time point a Turing machine machine is in one of a finite number of internal states. The internal state it is in determines how it behaves in response to a new symbol from the tape
    
The program implemented by a Turing machine consists of the rules that govern how it 

1. Moves between cells on the tape
2. Transitions between its internal states
3. Reads from and writes to the tape


## Example: Identifying Palindromes
One simple Turing machine program is the program which identifies whether a binary string is a palindrome. For our purposes, a palindrome is defined as an input for which the first half is a "mirror image" of the second half. For example, a string like "ABBA" is a palindrome, while "ABAB" is not.

### Input tape
We will provide input to the TM in the form of a Python `list` of symbols from our alphabet, where each element in the list represents a different cell on the tape. We use a special symbol, $*$, to indicate the end of our input. In the following program, we assume that our alphabet consists only of symbols in the set $\{A, B, *\}$. 

For example, if we wanted to evaluate whether the string `ABABBA` was a palindrome, we would provide our Turing machine with a tape of the form

                                        ['A', 'B', 'A', 'B', 'B', 'A', '*']
                                        
### Palindrome Program
Under these assumptions, we can represent the palindrome recognition program compactly using the following table:

|                  	|  Tape Symbol A  	|    Tape Symbol B   	|   Tape Symbol *     |
|:----------------:	|:----------:	|:-------------:	| :-------------:	|
| Internal State $i$ 	| ($p_0$, * , right) 	|   ($p_1$, * , right)  	| ($t$, * , right) |
| Internal State $p_0$ 	| ($p_0$, A, right) 	|   ($p_0$, B, right)  	| ($q_0$, * , left) |
| Internal State $p_1$ 	| ($p_1$, A, right) 	| ($p_1$, B, right) 	| ($q_1$, * , left) |
| Internal State $q_0$ 	| ($r$, * , left) 	| - 	| ($t$, * , right) |
| Internal State $q_1$ 	| -	| ($r$, * , left)  	| ($t$, * , right) |
| Internal State $r$ 	| ($r$, A, left) 	| ($r$, B, left) 	| ($i$, * , right) |


The contents of cell $(j,k)$ indicates the behavior of the machine when it is internal state $j$ and sees tape symbol $k$. For example, if the machine is in internal state `p_0` and reads the symbol `1` from the current tape cell, it executes the tuple ($p_0$, B, right). 
  - The first value of the tuple indicates the new internal state the Turing machine should transition to. 
  - The second value indicates the symbol the machine should write to the current cell. 
  - The third value indicates whether the tape head should move one cell the right or to the left once it is done writing to the current cell

We can implement this program in Python using simple `if` and `elif` statements. To make debugging a bit easier, we can assign the following names to each of the states in the table above:

$$
i \rightarrow \texttt{initial state} \\
p_0 \rightarrow \texttt{start was A; searching for the end} \\
p_1 \rightarrow \texttt{start was B; searching for the end} \\
q_0 \rightarrow \texttt{check that last symbol is A} \\
q_1 \rightarrow \texttt{check that last symbol is B} \\
r \rightarrow \texttt{returning to the beginning} \\
t \rightarrow \texttt{terminal state}
$$




An example implementation of this program in Python is below:

In [ ]:
def palindrome(state, symbol):
    i = "initial state"
    p_0 = "start was A; searching for the end"
    p_1 = "start was B; searching for the end"
    q_0 = "check that last symbol is A"
    q_1 = "check that last symbol is B"
    r = "returning to the beginning"
    t = "terminal state"
    
    if state == i and symbol == "A":
        return (p_0, '*', "right")
    elif state == i and symbol == "B":
        return (p_1, '*', "right")
    elif state == i and symbol == '*':
        return (t, '*', 'halt')
    
    elif state == p_0 and symbol == "A":
        return (p_0, "A", "right")
    elif state == p_0 and symbol == "B":
        return (p_0, "B", "right")
    elif state == p_0 and symbol == "*":
        return (q_0, "*", "left")
        
    elif state == p_1 and symbol == "A":
        return (p_1, "A", "right")
    elif state == p_1 and symbol == "B":
        return (p_1, "B", "right")
    elif state == p_1 and symbol == "*":
        return (q_1, "*", "left")

    elif state == q_0 and symbol == "A":
        return (r, "*", "left")
    elif state == q_0 and symbol == "*":
        return (t, "*", "halt")
    
    elif state == q_1 and symbol == "B":
        return (r, "*", "left")
    elif state == q_1 and symbol == "*":
        return (t, "*", "halt")
    
    if state == r and symbol == "A":
        return (r, 'A', "left")
    elif state == r and symbol == "B":
        return (r, 'B', "left")
    elif state == r and symbol == '*':
        return (i, '*', 'right')
    
    else:
        return (state, symbol, "halt")

### Running the Turing machine

On problem 4 in ps1 we provide you with a helper function called `turing_machine` which runs a Turing machine according to the rules of a program passed as an argument. The function is reproduced below: 

In [ ]:
def turing_machine(tape, program, verbose=False):
    """Run a given Turing machine with the given input.

    The program, accepted as the second argument, should accept two parameters:

      * state: A string representing the current state of the Turing machine.
        For example, the initial state that the Turing machine starts in is
        a string "initial state".
      * symbol: The current symbol being read on the tape, for example,
        "a" or "b". If there is nothing to read, then the symbol will be an
        empty string.

    The program should return a 3-tuple of (new state, new symbol, action):

      * new state: The next state of the Turing machine, as determined by
        your program.
      * new symbol: A new symbol to write on the tape *before* the Turing
        machine executes the action.
      * action: The action that the Turing machine should take after writing
        the new symbol. This should be a string, and should be either "left"
        (move left), "right" (move right), or "halt" (stop executing).

    The program is always run starting from the beginning (index 0) of the tape.
    If the Turing machine is at index 0, and the program returns an action of
    "left", then the tape will be extended to the left. If the Turing machine is
    at the last index of the tap, and the program returns an action of "right",
    then the tape will be extended to the right. In both cases, the symbol that
    is read after extending the tape is always an empty string.
    
    Parameters
    ----------
    tape: list
        A list of symbols representing the tape
    program: function
        The Turing machine to be run. Should accept two paramters:
        the current state of the Turing machine, and the current symbol
        being read.
    verbose: boolean (optional)
        Whether to print out the current state of the Turing machine
        after each step.
        
    Returns
    -------
    The symbol being read by the Turing machine when it halted.
        
    """
    
    # start in the intial state with the head over the first position
    state = "initial state"
    head = 0
    action = ""
    
    # helper function to print out the current state of the
    # Turing machine
    def print_tape(tape, head, state):
        print("state: " + str(state))
        tape = [str(x) if x != "" else " " for x in tape]
        print(" ".join(tape))
        print((" " * head * 2) + "^")
    
    # don't actually loop forever, to prevent infinite for loops
    for i in range(1000):
        if verbose:
            print_tape(tape, head, state)
        
        # run the next step of the program, which tells us the new
        # state, the new symbol, and what action to take
        state, tape[head], action = program(state, tape[head])
        
        # if the action is left, then make sure there is enough
        # tape on the left size
        if action == "left":
            if head == 0:
                tape.insert(0, "")
            else:
                head -= 1
                
        # if the action is right, then make sure there is enough
        # tape on the right side
        elif action == "right":
            if head == (len(tape) - 1):
                tape.append("")
            head += 1
            
        # if the action is halt, then stop
        elif action == "halt":
            break
            
        # unrecognized action
        else:
            raise ValueError("invalid action: " + str(action))

    if verbose:
        print_tape(tape, head, state)

    # return the last symbol
    if i == 1000:
        return None
    else:
        return tape[head]


### Halting Behavior
The `palindrome` program is designed to always halt (stop) in a finite number of timesteps. The state of the tape once it halts will allow us to determine whether the input was a palindrome. There are two possibilities:

 1. If our TM halts in its terminal state after having replaced all of the input symbols with our special $*$ symbol, we can conclude the input was a palindrome. 
 2. If instead our TM halts in anything other than its terminal state (i.e., with any non-$*$ symbols remaining on the tape), we can conclude the input was NOT a palindrome.

Below, we demonstrate how to run the `palindrome` program using the `turing_machine` helper function. Notice that we set the `verbose` flag to `True` in order to see the Turing machine's output at each step. This is very helpful when debugging!

In [ ]:
tape = ["A", "B", "B", "A", "*"] # test turing machine on a valid palindrome
turing_machine(tape, palindrome, verbose=True)

### Program Operation
Observe how this program operates: it implements a recursive solution which moves the up and down the tape, on each pass reducing the size of the problem it has to solve. In particular, if the program detects that the first and last letters (i.e., non-$*$ symbols) on the tape match, it replaces them both with the special $*$ symbol and then reruns the program on the modified tape. It continues doing this until either 

<ul>
  <li> It finds a case where the first and last letters do not match, indicating that the original string was not a palindrome</li>
  <li> The tape is filled with only $*$ symbols, indicating that the original string was a palindrome</li>
</ul>
<br>

<div class="alert alert-warning">
    Importantly, notice that the Turing machine program uses the $*$ symbol to mark which letters it has already checked. In this way, it uses the tape as a **memory**, allowing it to track its progress as it parses the input.
</div>

Finally, to get some more intuition about the program the machine implements, we could try translating it into a higher-level language like Python. As a Python program, a similar solution might look like:

In [ ]:
def palindrome_python(tape):    
    # strip all starting and trailing '*' symbols from tape
    remaining = tape.strip('*')
    
    # if the tape consists of only '*'
    if len(remaining) == 0:
        return tape
    
    # if the first and last non-* symbols match
    if remaining[0] == remaining[-1]:
        
        # replace first and last symbol non-* symbols on the tape with '*'
        tape = tape.replace(remaining[0], '*', 1)
        tape = tape[::-1].replace(remaining[0], '*', 1)[::-1]
        
        # print the updated tape
        print(tape)
        
        # run program on the updated tape
        return palindrome_python(tape)
    
    # if the first and last non-* symbols do not match
    else:
        return tape

In [ ]:
tape = 'ABBA*' # test turing machine on a valid palindrome
palindrome_python(tape)

---

Before turning this problem in remember to do the following steps:

1. **Restart the kernel** (Kernel$\rightarrow$Restart)
2. **Run all cells** (Cell$\rightarrow$Run All)
3. **Save** (File$\rightarrow$Save and Checkpoint)

<div class="alert alert-danger">After you have completed these three steps, ensure that the following cell has printed "No errors". If it has <b>not</b> printed "No errors", then your code has a bug in it and has thrown an error! Make sure you fix this error before turning in your problem set.</div>

In [ ]:
print("No errors!")